In [2]:
!pip install --upgrade xlrd

In [2]:
import pandas as pd
import numpy as np
import math
from scipy.integrate import quad

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def f1(sigma,t):   # t is the timestamp
  result = 1
  for k in range(20*math.floor((t-1)/20)+1, t+1):  # we want it to go from start ... t, that's why used t+1                  
    i = k-1  # id of data point, since k is id+1 (k is in column no.0 which is id+1)
    val = math.exp(-df[1][i]*df[1][i] / (2*sigma*sigma))
    val = val / (math.sqrt(2*math.pi)*sigma)
    result = result * val
  return result

In [4]:
def f2(t):
  result = 1
  for k in range(20*math.floor((t-1)/20)+1, t+1):                                   # 
    i = k-1
    val = math.pi*(1 + df[1][i]*df[1][i])
    val = 1 / val
    result = result * val
  return result

In [5]:
def m1(t):
  val = lambda t : quad(f1, 0.1,2, args=(t,))
  ans,err = val(t)
  return (1/1.9)*ans

In [6]:
def m2(t):
  return f2(t)

In [7]:
Pm1 = 0.5
Pm2 = 0.5

In [8]:
def Pm1Xt(t):
  val = Pm1 * m1(t)
  val = val / (Pm1*m1(t) + Pm2*m2(t))
  return val

In [9]:
def Pm2Xt(t):
  val = Pm2 * m2(t)
  val = val / (Pm1*m1(t) + Pm2*m2(t))
  return val

In [10]:
def sigmaapprox(t):
  start = 20*math.floor((t-1)/20)+1                                                 
  end = t
  if start==t: 
    return 0  # no variance if there is only one data point
  result = 1 / (end-start+1)
  val = 0
  for k in range(start,end+1):
    i = k-1
    val += df[1][i]*df[1][i]
  result = result * val
  result = math.sqrt(result)
  
  return result

In [11]:
def pg(t):
  sigma = sigmaapprox(t)
  if sigma==0:   # sigma = 0 means that we do not have any data yet to calculate sigma
    return 0.0001  # very little probability of hitting target
  def integrand(x):
      result = 1 / (sigma*math.sqrt(2*math.pi))
      result = result * math.exp(-x*x / (2*sigma*sigma))
      return result
  phi,err = quad(integrand,-float('inf'),-4)
  result = 1 - 2*(phi)
  if result>=1: 
    result = 0.999   # near certain of hitting the target
  return result 

In [12]:
def pc():
  return 0.844

In [13]:
def Cfunc(theta,t):
  C = 0
  for i in range(0,t%20):
    C+= pow(theta,i)
  return C

In [14]:
def DAfunc(kappa1 , kappa2 , theta ):
  DA_list =[]
  C = 0
  for i in range(0,20):
    if i == 0:
      DA_list.append(0)
      continue
    C = Cfunc(theta,i)
    DA_list.append(round(max(0, -kappa2 + 1/(1+math.exp(-kappa1*C))),3))
  return DA_list

In [15]:
def compute_expValue(DA,beta,alpha1,alpha2,alpha3,lamda,t):
  if t%20==0 or t%20==1: return 0   # t is the first or the second data point of a trial set
  Pg = pg(t)
  Pc = pc()
  a = np.exp(-(-math.log(Pg))**alpha3)
  b = lamda*np.exp(-(-math.log(1-Pg))**alpha3)
  c = np.exp(-(-math.log(Pc))**alpha3)
  d = lamda*np.exp(-(-math.log(1-Pc))**alpha3)
  V = Pm1Xt(t-1)*(a*pow(2+DA,alpha1) - b*pow(40,alpha2)) + Pm2Xt(t-1)*(c*pow(2+DA,alpha1) - d*pow(40,alpha2))
  return V # expected value

In [16]:
def predict(DA,beta,alpha1,alpha2,alpha3,lamda,starting,ending):
  total = 0
  plist = []  # list of outcomes (bet or not bet)
  expList = []  # list of expected values for each time stamp
  probList = []
  probBaseList = []
  for i in range(starting,ending): 
    # i is the time stamp. 
    Vt = compute_expValue(DA[i%20],beta,alpha1,alpha2,alpha3,lamda,i) # Computing expected value for timestamp i
    prob_base = math.exp(beta*Vt) / (1 + math.exp(beta*Vt))
    prob = DA[i%20]+(1-DA[i%20])*prob_base
    expList.append(Vt)
    probBaseList.append(prob_base)
    probList.append(prob)
    if prob>0.5:
      plist.append(1)
      if df[1][i]<=4 and df[1][i]>=-4: # in the dataframe, index of data at time i+1 is i
        total+=2 
      else: 
        total-=40
    else: plist.append(0)        # adding value for timestamp i+1
  return (plist,expList,probList,probBaseList,total)   

In [116]:
df = pd.read_excel(r'C:\Users\Hitanshee\OneDrive\Documents\BlackcofferFiles\DATA_AJAY\DATA_AJAY\DATA2\DATA_Experiment3\8.xls',sheet_name='results',header=None)
df
Y = df[4]

In [18]:
def d_C(theta, t):
  C = 0
  for i in range(1,t%20+1):
    C += i*pow(theta,i-1)
  return C

In [19]:
def derivative_cost_function(kappa1,kappa2,theta,beta,alpha1,alpha2,alpha3,lamda):  # gradient of cost function (used in gradient descent)
  G_a1 = 0   # total gradient 
  G_a2 = 0
  G_a3 = 0
  G_L = 0
  G_k1 = 0
  G_k2 = 0
  G_t = 0
  G_b = 0
  plist,expList,probList,probBaseList,total = predict(DAfunc(kappa1,kappa2,theta),beta,alpha1,alpha2,alpha3,lamda,0,160)
  DA = DAfunc(kappa1,kappa2,theta)
  for i in range(1,160):  
    if (i+1)%20<5: continue
    pm1 = Pm1Xt(i-1)
    pm2 = Pm2Xt(i-1)
    Pg = pg(i)
    Pc = pc()
    C = Cfunc(theta,i)
    d_a1 = pm1*(np.exp(-(-math.log(Pg))**alpha3)*(2+DA[i%20])**alpha1*math.log(2+DA[i%20])) + pm2*(np.exp(-(-math.log(Pc))**alpha3)*(2+DA[i%20])**alpha1*math.log(2+DA[i%20]))          #Derivative(V,a1)
    d_a2 = -pm1*lamda*(np.exp(-(-math.log(1-Pg))**alpha3)*40**alpha2) - pm2*lamda*(np.exp(-(-math.log(1-Pc))**alpha3)*40**alpha2)
    d_a3 = pm1*(-(-math.log(Pg))**alpha3*math.log(-math.log(Pg))*np.exp(-(-math.log(Pg))**alpha3)*(2+DA[i%20])**alpha1 + lamda*(-math.log(1-Pg))**alpha3*math.log(-math.log(1-Pg))*np.exp(-(-math.log(1-Pg))**alpha3)*40**alpha2) + pm2*(-(-math.log(Pc))**alpha3*math.log(-math.log(Pc))*np.exp(-(-math.log(Pc))**alpha3)*(2+DA[i%20])**alpha1 + lamda*(-math.log(1-Pc))**alpha3*math.log(-math.log(1-Pc))*np.exp(-(-math.log(1-Pc))**alpha3)*40**alpha2)
    d_L = pm1*np.exp(-(-math.log(1-Pg))**alpha3)*40**alpha2 + pm2*np.exp(-(-math.log(1-Pc))**alpha3)*40**alpha2
    d_k1 = (pg(i-1)*Pm1Xt(i-2) + pc()*Pm2Xt(i-2))*((C*math.exp(-kappa1*C))/((1+math.exp(-kappa1*C))*(1+math.exp(-kappa1*C))))
    d_k2 = (pg(i-1)*Pm1Xt(i-2) + pc()*Pm2Xt(i-2))*(-1)
    d_t = (pg(i-1)*Pm1Xt(i-2) + pc()*Pm2Xt(i-2))*((kappa1*math.exp(-kappa1*C))/((1+math.exp(-kappa1*C))*(1+math.exp(-kappa1*C))))*d_C(theta,i)

    G_a1 += (probList[i]-Y[i])*beta*d_a1/160
    G_a2 += (probList[i]-Y[i])*beta*d_a2/160
    G_a3 += (probList[i]-Y[i])*beta*d_a3/160
    G_L += (probList[i]-Y[i])*beta*d_L/160
    G_k1 += (probList[i]-Y[i])*beta*d_k1
    G_k2 += (probList[i]-Y[i])*beta*d_k2
    G_t += (probList[i]-Y[i])*beta*d_t
    G_b += (probList[i]-Y[i])/(probList[i]*(1-probList[i]))*(1-DA[i%20])*probBaseList[i]*(1-probBaseList[i])*compute_expValue(DA[i%20],beta,alpha1,alpha2,alpha3,lamda,i)
  
  G_a1 /= 160
  G_a2 /= 160
  G_a3 /= 160
  G_L /= 160 
  G_k1 /= 160
  G_k2 /= 160
  G_t /= 160
  G_b /=160
  return (G_k1,G_k2,G_t,G_b,G_a1,G_a2,G_a3,G_L)

In [117]:
kappa1 = 0.5
kappa2 = 0.7
theta = 0.9
beta = 1
alpha1,alpha2,alpha3,lamda = 1.0,1.0,1.0,1.0
alpha = 0.1
i = 0
while True:
  if i==100: break
  if i==0:
    pre_k2,pre_beta,pre_a1,pre_a2,pre_a3,pre_L = kappa2,beta,alpha1,alpha2,alpha3,lamda
    i+=1
  else:
    G_k1,G_k2,G_t,G_b,G_a1,G_a2,G_a3,G_L = derivative_cost_function(kappa1,kappa2,theta,beta,alpha1,alpha2,alpha3,lamda)
    temp1 = kappa1 - alpha*G_k1
    temp2 = kappa2 - alpha*G_k2
    temp3 = theta - alpha*G_t
    beta = beta - alpha*G_b
    temp4 = alpha1 - alpha*G_a1
    temp5 = alpha2 - alpha*G_a2
    temp6 = alpha3 - alpha*G_a3
    temp7 = lamda - alpha*G_L
    kappa1,kappa2, theta = temp1,temp2,temp3
    alpha1,alpha2,alpha3,lamda = temp4,temp5,temp6,temp7
    if alpha1>1: alpha1 = 1
    if alpha2>1: alpha2 = 1
    if alpha3>1: alpha3 = 1
    if lamda<=1: lamda = 1.01
    if kappa1>1: kappa1 = 0.99
    if kappa1<0.1: kappa1 = 0.1
    if theta>1: theta = 0.99
    if theta<0.5: 
      theta = 0.5
      break
    if kappa2<0.2: kappa2 = 0.2
    if kappa2 > 1: 
      kappa2 = 0.99
      break
    if beta<0: beta = 0.1
    if abs(kappa2-pre_k2)<1e-3 and abs(beta-pre_beta)<1e-3: break
    else: 
      pre_k2,pre_beta,pre_a1,pre_a2,pre_a3,pre_L = kappa2,beta,alpha1,alpha2,alpha3,lamda
      pre_k2 = kappa2
    # abs(alpha1-pre_a1)<1e-3 or abs(alpha2-pre_a2)<1e-3 or abs(alpha3-pre_a3)<1e-3 or abs(lamda-pre_L)<1e-3 
    print(str(i)+"kappa1, kappa2, theta, beta: ",kappa1,kappa2,theta,beta)
    print("alpha1, alpha2, alpha3, lamda: ",alpha1,alpha2,alpha3,lamda)
    i+=1
print("kappa1, kappa2, theta, beta: ",kappa1,kappa2,theta,beta)
print("alpha1, alpha2, alpha3, lamda: ",alpha1,alpha2,alpha3,lamda)

1kappa1, kappa2, theta, beta:  0.4904939811809482 0.7407828783036677 0.8794279870244408 0.9518145207494326
alpha1, alpha2, alpha3, lamda:  0.9995319722584527 1 0.9993593581397814 1.01
2kappa1, kappa2, theta, beta:  0.48022002815928866 0.777202609385828 0.8576727313199528 0.9067619353281067
alpha1, alpha2, alpha3, lamda:  0.9991357194084232 1 0.9988334850416467 1.0095934087798435
3kappa1, kappa2, theta, beta:  0.4693086160875396 0.809688985957326 0.8354975669514275 0.8646291246210448
alpha1, alpha2, alpha3, lamda:  0.9988009155149937 1 0.9984137484154141 1.009271872784887
4kappa1, kappa2, theta, beta:  0.4580822766685554 0.8385705770445616 0.8139314930066432 0.8253542070493993
alpha1, alpha2, alpha3, lamda:  0.998519286813812 1 0.9980886039814193 1.0090263783339848
5kappa1, kappa2, theta, beta:  0.4468616763622877 0.864344263325536 0.7937926380298775 0.788762836775458
alpha1, alpha2, alpha3, lamda:  0.9982809109375382 1 0.9978415035888374 1.0088439215587566
6kappa1, kappa2, theta, beta:

In [ ]:
plist,expList,probList,probBaseList,total = predict(DAfunc(kappa1,kappa2,theta),beta,alpha1,alpha2,alpha3,lamda,160,300)   # prediction

print(len(plist))
print(total)

In [ ]:
total = 0
for i in range(0,140):  # we will have to change it to (0,160) when the dataset has a length of 320
  if (plist[i]==0 and Y[i+160]==0) or (plist[i]==1 and Y[i+160]==1): total+=1
print("The accuracy score of the base model with CbD is: ",total/140*100,"%")   # similarly here 140 -> 160